# Web scraping

Cuando necesitamos extraer información publicada en internet, lo ideal es consultar una API, porque:

* Las respuestas contienen información estructurada
* En general, el propio servicio nos da documentación sobre cómo hacer peticiones y qué tipo de información podemos solicitar

Pero muchas veces nos encontramos con información en páginas web (en formato [HTML](https://es.wikipedia.org/wiki/HTML)) que nos gustaría obtener, pero sin API disponible.

Estas páginas `HTML` tienen cierta estructura, aunque con ciertos contras:

* Es más compleja, puede tener muchos niveles de anidamiento
* Es inestable. Están diseñadas para que se vean bien desde el explorador, no para guardar una estructura de consulta. De un día para otro, puede verse alterada por la incorporación de nuevos elementos visuales u otros motivos.
* Puede ser modificada por código cliente (javascript) en diferentes momentos: al cargar la página, al interaccionar con algún elemento, ...

## Scraping de elementos html

La librería que vamos a utilizar es [Beautiful Soup](https://pypi.org/project/beautifulsoup4/). Nos permite buscar elementos y navegar por la estructura del html fácilmente.

Vemos dos ejemplos, uno sobre milanuncios y otro sobre spotahome, por si nos _banean_.

### milanuncios

Imaginemos que queremos comparar precios de un determinado modelo de motocicleta de segunda mano. P.e. con [esta búsqueda](https://www.milanuncios.com/motos-de-carretera/duke-390.htm) en milanuncios.

La mayor parte de las webs con contenido interesante (que hacen negocio gracias a su contenido) intentan protegerlas para evitar que les hagan scraping. Hay varias formas de simular que nuestro script es humano en lugar de un bot, algunas más básicas y otras más complejas. Por ahora, vamos a sobrescribir nuestro _user agent_. Es una cabecera que va en las peticiones diciendo quiénes somos (p.e. qué tipo de explorador usamos). Por defecto, la librería `requests` que vamos a usar, avisa que somos un bot. Vamos a sobrescribir esta cabecera para _disimular_ un poco. Podemos copiar uno popular de (aquí)[https://developers.whatismybrowser.com/useragents/explore/software_type_specific/web-browser/].

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

Ahora, nos descargamos el html con `requests`.

In [3]:
page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', headers=headers)
page

<Response [200]>

Podemos ver el contenido examinando la propiedad `content`.

In [4]:
page.content

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="es" lang="es">\n<head><title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n    <meta name="robots" content="noarchive">\n        <meta name="description" content="Compra venta de motos de carretera de ocasi\xf3n duke 390. Todas las marcas: Aprilia, BMW, Cagiva, Derbi, Honda, Yamaha, Kawasaki, Suzuki">\n<link rel="shortcut icon" href="https://static.milanuncios.com/202004201415-master/favicon.ico">    <link rel="stylesheet" type="text/css" href="https://static.milanuncios.com/202004201415-master/css/estilos.css">\n    <link rel="canonical" href="https://www.milanuncios.com/motos-de-carretera/duke-390.htm" />\n<script type="text/javascript"><!--\nfunction prebu(c){return c;}\nvar esmipri=true;\nvar esmipri2=true;\n// -->\n</script>\n<script async="asyn

Este contenido es solo texto, no tiene estructura. Aún no podemos hacer búsquedas ni navegar por él.

Para hacerlo, creamos una instancia de `Beautiful Soup` y lo parseamos

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

#### SOS: ¡me han baneado!

Si estamos haciendo esto en clase, es muy probable que a la mayoría, nos detecten como bots y nos baneen. Esto pasa porque somos 30 o 40 personas, haciendo la misma petición desde la misma IP y con el mismo user agent.

Puedes usar el html offline que hay en `dat` para seguir con el ejercicio. Descomenta el siguiente código y sigue adelante

In [ ]:
# Descomenta esto si te han baneado
# with open('dat/milanuncios.html') as f:
#     soup = BeautifulSoup(f, 'html.parser')

Sobre esto, podemos hacer búsquedas con `find` y `find_all` (o `select_one` y `select` si prefieres utilizar [selectores css](https://en.wikipedia.org/wiki/Cascading_Style_Sheets#Selector)). Sobre nuestro ejemplo, vamos a buscar todos los precios. Examinando el código fuente, vemos que son etiquetas `div` con clase `aditem-price`.

In [7]:
div_precios = soup.find_all('div', class_='aditem-price')
div_precios

[<div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">3.999<sup>€</sup></div>,
 <div class="aditem-price">3.499<sup>€</sup></div>,
 <div class="aditem-price">3.199<sup>€</sup></div>,
 <div class="aditem-price">4.899<sup>€</sup></div>,
 <div class="aditem-price">2.999<sup>€</sup></div>,
 <div class="aditem-price">3.800<sup>€</sup></div>,
 <div class="aditem-price">4.390<sup>€</sup></div>,
 <div class="aditem-price">4.870<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">3.199<sup>€</sup></div>,
 <div class="aditem-price">3.200<sup>€</sup></div>,
 <div class="aditem-price">2.900<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">4.100<sup>€</sup></div>,
 <div class="aditem-price">4.999<sup>€</sup></div>,
 <div class="aditem-price">3.800<sup>€</sup></div>,
 <div class="aditem-price">2.950<sup>€</sup></div>,
 <div class="aditem-price">4.300<sup>€</sup></div>,
 <div class=

`find_all` devuelve una lista de elementos. Sobre ellos, podemos hacer:

`children` para sacar el listado de todos los hijos.

In [8]:
list(div_precios[0].children)

['3.500', <sup>€</sup>]

`get_text()` para sacar el texto de todos los hijos

In [9]:
div_precios[0].get_text()

'3.500€'

Por tanto, para sacar el listado de todos los precios podemos hacer:

In [10]:
[list(div_precio.children)[0] for div_precio in div_precios]

['3.500',
 '3.999',
 '3.499',
 '3.199',
 '4.899',
 '2.999',
 '3.800',
 '4.390',
 '4.870',
 '3.900',
 '3.199',
 '3.200',
 '2.900',
 '3.500',
 '4.100',
 '4.999',
 '3.800',
 '2.950',
 '4.300',
 '3.400',
 '3.400',
 '3.400',
 '3.400',
 '3.400',
 '4.600',
 '4.950',
 '1.000',
 '3.399']

Tienes más funciones útiles con pequeños ejemplos [aquí](http://akul.me/blog/2016/beautifulsoup-cheatsheet/)

## Scraping de tablas

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

Necesitaremos los siguientes módulos además de `requests` y `BeautifulSoup` importados anteriormente:

In [11]:
import pandas as pd
import re

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [12]:
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content

La siguiente línea procesa el HTML de la página que hemos descargado:

In [13]:
soup = BeautifulSoup(contenido, "html.parser")

Una vez procesado el HTML, es posible buscar elementos dentro de él. En particular, podemos buscar los elementos de tipo `table`, es decir, tablas.

In [14]:
tablas = soup.find_all('table')

El objeto `tablas` contiene todas las tablas presentes en la página. Hay que tener cuidado con dichas tablas porque muchas páginas utilizan elementos de tipo `table` para estructurar el contenido. Por eso, en algunas páginas, aunque parezca haber una única tabla, puede haber otras con una información no interesante que toca descartar.

In [ ]:
len(tablas)

Podemos extraer las filas de todas estas tablas

In [ ]:
lineas = [x for tabla in tablas for x in tabla.find_all('tr')]

para luego extraer los contenidos de cada fila individualmente haciendo

In [ ]:
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas]

Podemos inspeccionar parte del objeto resultante:

In [ ]:
datos[0:3]

Vemos que hay filas que contienen la información de interés junto con otras que contienen cabeceras y otra información irrelevante. En general, la situación puede ser más complicada y se hace necesario estudiar el objeto `tablas` para seleccionar la de interés.

En nuestro caso, podemos filtrar las líneas menos relevantes así:

In [ ]:
datos = [x for x in datos if len(x) > 0]

Finalmente, podemos crear una tabla de Pandas:

In [ ]:
datos = pd.DataFrame(datos)
datos

## Riesgos del scraping

El scraping es una técnica potente pero tiene varios contras:

* Implica mayor tiempo de desarrollo y mayor esfuerzo en la limpieza de datos (en comparación con otras fuentes como APIs, BDs, ...)
* Si hay que scrapear gran cantidad de páginas, es lento
* Los servidores objetivo de nuestro scraping pueden tener técnicas para evitarlo. Por ejemplo, bloquear la IP temporalmente o introducir delays en las respuestas si hacemos muchas peticiones en poco tiempo. Esto pasa especialmente en las grandes webs recelosas de sus datos (p.e. linkedin, amazon, ...).
* El código de scraping escrito hoy puede no funcionar mañana, si la web destino cambia nombres, etiquetas o estructura. Si se sube a producción para lanzarlo periódicamente, hay que ser conscientes de que en algún momento fallará, y establecer mecanismos de alerta

## Javascript

Es posible que te encuentres con algún caso en el que no puedas descargar tal cual el html y parsearlo, principalmente por dos motivos:

* La estructura de la página se genera parcial o totalmente en cliente
* Debemos interactuar con algún elemento para mostrar la información que queremos (p.e. completar un campo de búsuqeda, hacer click en algún botón, ...)

En estos casos, hay que ejecutar en un navegador local el código javascript de la página destino. Para esta tarea, puedes utilizar [Selenium]().

[Aquí](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251) un post con un ejemplo de uso.